<a href="https://colab.research.google.com/github/bttrung/secure-private-ai-scholarship/blob/master/9__Securing_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

W0825 02:00:46.523431 139676721686400 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0825 02:00:46.539638 139676721686400 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0825 02:00:47.415601 139676721686400 base.py:654] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0825 02:00:47.416740 139676721686400 base.py:654] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0825 02:00:47.421467 139676721686400 base.py:654] Worker bob already exists. Replacing old worker which could 

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [0]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss


In [0]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [0]:
with th.no_grad():

    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())


In [7]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0533) Alice:tensor(4.9797e-05)
Bob:tensor(0.0384) Alice:tensor(8.6088e-06)
Bob:tensor(0.0280) Alice:tensor(5.6922e-05)
Bob:tensor(0.0207) Alice:tensor(9.5141e-05)
Bob:tensor(0.0155) Alice:tensor(0.0001)
Bob:tensor(0.0116) Alice:tensor(0.0001)
Bob:tensor(0.0088) Alice:tensor(0.0001)
Bob:tensor(0.0067) Alice:tensor(9.1083e-05)
Bob:tensor(0.0051) Alice:tensor(7.6627e-05)
Bob:tensor(0.0039) Alice:tensor(6.2885e-05)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).sum()


In [9]:
print(preds)
print(target)
print(loss.data)


tensor([[0.1731],
        [0.1403],
        [0.8236],
        [0.7907]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.1246)


In [0]:
# Intro to Additive Secret Sharing
x = 5

In [12]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x


5

In [13]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x


10

In [14]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

In [15]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [16]:
(bob_x_share + alice_x_share) % Q

5

In [0]:
# Build Methods for Encrypt, Decrypt, and Add
x_share = (2,5,7)

In [0]:
import random

Q = 23740629843760239486723

def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

def decrypt(shares):
    return sum(shares) % Q


In [19]:
shares = encrypt(3)
shares


(6204025632784607086314, 22609768564978800559027, 18667465489757071328108)

In [20]:
decrypt(shares)

3

In [0]:
def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [22]:
x = encrypt(5)
y = encrypt(7)
z = add(x,y)
decrypt(z)

12

In [0]:
# Fixed Precision Encoding
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [25]:
encode(3.5)

35000

In [26]:
decode(35000)

3.5

In [27]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

In [0]:
# Secret Sharing + Fixed Precision in PySyft
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.share(bob, alice, secure_worker)

In [31]:
bob._objects

{43092931334: tensor([3176685041567022792, 4302081302991596467, 3589858796618381312,
         2005990585198494219, 1642572013280612609])}

In [33]:
y = x + x
y

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:9946190295 -> bob:31676442599]
	-> [PointerTensor | me:47405004649 -> alice:59837844553]
	-> [PointerTensor | me:77174130334 -> secure_worker:77732426009]
	*crypto provider: me*

In [34]:
y.get()

tensor([ 2,  4,  6,  8, 10])

In [40]:
# Fixed Precision using PySyft
x = th.tensor([0.1,0.2,0.3])
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()

In [42]:
x.child.child

tensor([100, 200, 300])

In [0]:
y = x + x

In [44]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

In [0]:
# Shared Fixed Precision
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [0]:
y = x + x

In [48]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])